# 1 -  Planejamento da Solução

## Entendimento de Negócio

**Qual é o problema de negócio?**

- Empresa alimentícia situada no RJ, deseja abrir filiais na cidade de São Paulo. Para isso, precisa de 3 análises:
    - 1 - Estimar faturamento que uma loja teria em cada um dos bairros de São Paulo (regressão, neste caso sem séries temporais).
    - 2 - Classificar o potencial dos bairros de São Paulo em alto, médio ou baixo (classificação multiclasse).
    - 3 - Segmentar os bairros de São Paulo de acordo com o perfil de renda e idade, identificando os com maior aderência ao público alvo.
    
- Público alvo: adultos de 25 a 50 anos, das classes A (rendas A1 e A2) e B (rendas B1 e B2). 

**Saída**

- Prototipagem técnica da solução: https://docs.google.com/spreadsheets/d/17lxCBRLPEuNCO25BimVFRE3Tms-314WSTpJ1xgEIf38/edit#gid=0

- O que será entregue, efetivamente? / Onde o time de negócio quer ver?
    - Documento no formato doc, pdf ou ppt, voltado ao negócio, apresentando um racional de como os dados foram analisados. Detalhar com gráficos, tabelas, e descrever conclusões.
    - Responder: Dada a natureza do problema apresentado, que outro dado externo (fontes públicas ou privadas) poderia ser utilizado para agregar mais valor ao resultado? Por que?
    
**Entrada**
- Fontes de dados:
    - Dataset contendo faturamento e potencial dos bairros do Rio de Janeiro do cliente, bem como dados sociodemográficos do bairros do Rio de Janeiro e São Paulo.

- Ferramentas:
    - Python 3.8.12, Jupyter Notebook, Git, Github.

**Processamento**
- Tipo de problema: análise exploratória, regressão, classificação e clusterização.
- Metodologia: CRISP-DM, metodologia ágil (iterativa e incremental) para desenvolvimento de projetos de ciência de dados.


## Implementado nesta Sprint

**Ciclo 3**

Desenvolvimento da Análise 1 (regressão):
- Split dos dados.
- Modelagem de dados (data preparation).
- Seleção de Features.
- Model Based feature selection.
- Novo KNN Regressor + comparação com o baseline do ciclo 1.
- KNN Regressor com Random Search + Cross validation.
- Novos algoritmos de ML: Lasso (linear), XGBoost Reegressor (embedding de trees).
- Comparação da performance dos modelos.
- Performance de generalização do melhor modelo com dados de teste.
- Modelos final, treinado com dataset traino + validação, prevendo faturamento por bairro em SP (produção).

# 2 - Importações

## Bibliotecas

In [350]:
import pandas                      as pd 
import seaborn                     as sns
import numpy                       as np
import sweetviz                    as sv
import xgboost                     as xgb
import inflection
import warnings

from IPython.core.display          import HTML
from matplotlib                    import pyplot as plt
from tabulate                      import tabulate
from matplotlib.ticker             import FuncFormatter
from IPython.display               import Image

from sklearn.model_selection       import train_test_split, GridSearchCV
from sklearn.neighbors             import KNeighborsRegressor
from sklearn.metrics               import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing         import MinMaxScaler, OrdinalEncoder
from sklearn.feature_selection     import SelectFromModel, SelectKBest, f_regression
from sklearn.ensemble              import RandomForestRegressor
from sklearn.linear_model          import Lasso

## Funções Auxiliares

In [2]:
def jupyter_settings():
    """ Otimiza configurações gerais, padronizando tamanhos de plots, etc """
    %matplotlib inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [20, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.set_option('display.max_columns', 30)
    pd.set_option('display.max_rows', 30)
    pd.set_option('display.expand_frame_repr', False )
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    pd.set_option('max_colwidth', None)
    sns.set()
jupyter_settings()

def ml_error( model_name, y, yhat ):
    """ Calcula os erros do modelo de regressão recebido 
    model_name: nome do modelo
             y: valores de reais
          yhat: valores de estimados pelo modelo """
    mae = mean_absolute_error (y, yhat)
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    
    return pd.DataFrame( {'Model Name': model_name,
                          'MAE': mae,
                          'MAPE': mape,
                          'RMSE': rmse  }, index=[0] )

# 3 - Análise 1: Estimativa de faturamento em SP

Análise 1 - Estimar o faturamento que uma loja teria em cada um dos bairros de São Paulo (regressão).

## Split dos Dados

In [81]:
df_rj = pd.read_csv('../data/interim/df_rj_feat_eng_done.csv', index_col=0)
print(df_rj.shape)
df_rj.head(1)

(160, 22)


,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,faturamento,potencial
0,11676,1027,483,688,800,1675,2300,1784,2919,3975,0,145,715,1242,1093,758,92,304,2102,2501.00,932515,Médio


A análise 1 é um problema de regressão (supervisionado), sendo a variável alvo "faturamento" do tipo float (contínua).

df_rj será dividida em três partes, sendo:
- Dados de treinamento (train): utilizados para treinar os modelos de machine learning.
- Dados de validação (val): utilizados para validar a performance dos modelos, e para tunagem de hiperparâmetros.
- Dados de teste (test): utilizados para avaliação final da capacidade de generalização dos modelos, simulando dados de produção (registros de SP).

Divisão entre features e variável alvo:

In [82]:
y1 = df_rj.faturamento
y1.head(2)

0    932515
1    588833
Name: faturamento, dtype: int64

In [83]:
X1 = df_rj.drop(["faturamento"], axis=1)
X1.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
0,11676,1027,483,688,800,1675,2300,1784,2919,3975,0,145,715,1242,1093,758,92,304,2102,2501.00,Médio


Divisão entre datasets de treino/validação e teste, mantendo 15% dos registros para teste.

O parâmetro "random_state" é utilizado para manter a reprodutibilidade da divisão.

In [84]:
X1_trainval, X1_test, y1_trainval, y1_test = train_test_split(X1, y1, random_state=0, test_size=0.15)

Divisão entre datasets de treino e validação (a partir do trainval), mantendo 15% dos registros para validação.

In [85]:
X1_train, X1_val, y1_train, y1_val = train_test_split(X1_trainval, y1_trainval, random_state=0, test_size=0.15)

Conferência das divisões:

In [86]:
print(X1_train.shape)
print(X1_val.shape)
print(X1_test.shape)
X1_train.head(1)

(115, 21)
(21, 21)
(24, 21)


,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
63,16177,1516,748,1016,1174,2488,3497,2327,3411,5985,0,141,857,1464,1740,1030,87,432,2462,2336.00,Médio


## Preparação dos Dados

Os algoritmos de machine learning possuem premissas com relação ao formato dos dados que recebem. Elas devem ser respeitadas sempre que possível, para que o algoritmo atinja a maior performance possível.

Nesta seção, as features serão preparadas para servir de insumo aos algoritmos.

In [87]:
X1_train.columns

Index(['populacao', 'pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24',
       'pop_de25a34', 'pop_de35a49', 'pop_de50a59', 'pop_mais_de60',
       'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1',
       'domicilios_b2', 'domicilios_c1', 'domicilios_c2', 'domicilios_d',
       'domicilios_e', 'domicilios_alvo', 'renda_media', 'potencial'],
      dtype='object')

In [88]:
X1_train.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
63,16177,1516,748,1016,1174,2488,3497,2327,3411,5985,0,141,857,1464,1740,1030,87,432,2462,2336.00,Médio


Tratativas necessárias para as features da análise 1 (regressão).

- Numéricas (tratadas com scaling): 'populacao', 'pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24', 'pop_de25a34', 'pop_de35a49', 'pop_de50a59', 'pop_mais_de60', 'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1', 'domicilios_b2', 'domicilios_c1', 'domicilios_c2', 'domicilios_d', 'domicilios_e', 'domicilios_alvo', e 'renda_media'. 
- Categóricas (tratadas com encoding): 'potencial'.
- Variável resposta (tratada com transformação logaritmica): 'faturamento'.

### Scaling

1 - Feature Scaling: Trazer todas variáveis para mesmo range (escala), para não enviesar os modelos.
- 1.1 Standardization se distribuição Normal:
    - Sem outliers: usa StandardScaler
    - Com outliers: usa RobustScaler
- 1.2 Normalization se não normal:
    - Com ou sem outliers: Usar MinMaxScaler

Avaliando as distribuições das features na seção "Estatística Descritiva", nenhuma das que serão tratatas aqui possui distribuição normal. Logo, todas serão reescaladas usando a classe "MinMaxScaler" do scikit-learn.
- MinMaxScaler: Rescala para o intervalo entre 0 e 1.

In [89]:
features_scaling = ['populacao', 'pop_ate9', 'pop_de10a14', 'pop_de15a19', 'pop_de20a24', 'pop_de25a34',
'pop_de35a49', 'pop_de50a59', 'pop_mais_de60', 'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1',
'domicilios_b2', 'domicilios_c1', 'domicilios_c2', 'domicilios_d', 'domicilios_e', 'domicilios_alvo', 'renda_media']

Reescala apenas as features selecionadas do treino com MinMaxScaler.

In [90]:
scaler1 = MinMaxScaler()
X1_train[features_scaling] = scaler1.fit_transform(X1_train[features_scaling])
X1_train.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
63,0.02,0.01,0.01,0.02,0.02,0.03,0.03,0.03,0.04,0.03,0.00,0.01,0.04,0.07,0.05,0.04,0.02,0.03,0.05,0.03,Médio


Aplica o scaler já treinado apenas nas features selecionadas de validação.

In [91]:
X1_val[features_scaling] = scaler1.transform(X1_val[features_scaling])
X1_val.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
55,0.03,0.02,0.02,0.02,0.03,0.04,0.04,0.02,0.02,0.04,0.00,0.01,0.01,0.04,0.06,0.09,0.08,0.04,0.02,0.02,Baixo


Já deixar pronto também trainval, para avaliar a capacidade de generalização no final.

In [92]:
scaler1_tv = MinMaxScaler()
X1_trainval[features_scaling] = scaler1.fit_transform(X1_trainval[features_scaling])
X1_trainval.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
37,0.07,0.05,0.05,0.06,0.06,0.08,0.09,0.07,0.09,0.09,0.00,0.00,0.05,0.11,0.14,0.16,0.15,0.18,0.07,0.01,Médio


Aprontar também o test, já aplicando a reescala.

In [93]:
X1_test[features_scaling] = scaler1_tv.transform(X1_test[features_scaling])
X1_test.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
110,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.00,0.01,0.02,0.03,0.03,0.04,0.04,0.06,0.02,0.02,Baixo


Features Reescaladas.

### Transformations

2 - Transformação:
- 2.1 - Converter variáveis categóricas em numéricas (Encoding).
- 2.2 - Converter a variável resposta em distribuição normal.(Response Variable Transf.)

#### Encoding

2.1 - Encoding:
- É a Conversão de Features Categóricas para Numéricas, mantendo o conteúdo.
- Avaliando a feature "potencial", há uma relação de ordem na variável, que contém valores "Baixo", "Médio" e "Alto". Em função disso, será aplicando um encoding ordinal através da classe "OrdinalEncoder" do scikit-learn.

Encodar apenas as features selecionadas do treino com OrdinalEncoder.

In [96]:
encoder1 = OrdinalEncoder(categories=[['Baixo', 'Médio', 'Alto']], dtype=int)
X1_train.potencial = encoder1.fit_transform(X1_train.potencial.to_numpy().reshape(-1,1))
encoder1.categories_

[array(['Baixo', 'Médio', 'Alto'], dtype=object)]

Aplica o encoder já treinado apenas na feature selecionada de validação.

In [101]:
X1_val.potencial = encoder1.transform(X1_val.potencial.to_numpy().reshape(-1,1))
X1_val.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
55,0.03,0.02,0.02,0.02,0.03,0.04,0.04,0.02,0.02,0.04,0.00,0.01,0.01,0.04,0.06,0.09,0.08,0.04,0.02,0.02,0


Já deixar pronto também trainval, para avaliar a capacidade de generalização no final.

In [103]:
encoder1_tv = OrdinalEncoder(categories=[['Baixo', 'Médio', 'Alto']], dtype=int)
X1_trainval.potencial = encoder1.fit_transform(X1_trainval.potencial.to_numpy().reshape(-1,1))
X1_trainval.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
37,0.07,0.05,0.05,0.06,0.06,0.08,0.09,0.07,0.09,0.09,0.00,0.00,0.05,0.11,0.14,0.16,0.15,0.18,0.07,0.01,1


Aprontar também o test, já aplicando as transformações.

In [105]:
X1_test.potencial = encoder1.transform(X1_test.potencial.to_numpy().reshape(-1,1))
X1_test.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media,potencial
110,0.02,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.00,0.01,0.02,0.03,0.03,0.04,0.04,0.06,0.02,0.02,0


#### Response Variable Transformation

2.2 - Response Variable Transformation:
- Aplica um log na variável resposta, transformando ela em algo próximo a uma distribuição normal.

Será tratado em ciclos futuros. 

## Seleção de Features

Serão criados diversos datasets com features distintas, através de diferentes métodos de seleções de feature.

O objetivo é realizar experimentações em um mesmo modelo, mantendo o dataset com as features com maior performance.

### Baseada na EDA

Uma forma simples de selecionar as features, é escolher as com maior correlação posisitva com a variável alvo. 

Estas podem ser vistas no relatório do SweetViz da análise 1, exibido na EDA.

Mantendo apenas as features selecionadas em "X1_train" "X1_val". 

In [118]:
X1_train_eda = X1_train[['domicilios_a1', 'domicilios_a2', 'domicilios_b2','renda_media','potencial']].copy()
X1_val_eda = X1_val[['domicilios_a1', 'domicilios_a2',  'domicilios_b2','renda_media','potencial']].copy()
X1_train_eda.head(1)

,domicilios_a1,domicilios_a2,domicilios_b2,renda_media,potencial
63,0.00,0.01,0.07,0.03,1


### Baseada no Público Alvo

Consiste em simplesmente eliminar as features relacionadas a população e renda, não relacionadas diretamente com o público alvo.

In [119]:
X1_train_pa = X1_train[['pop_alvo', 'domicilios_alvo', 'renda_media', 'potencial']].copy()
X1_val_pa = X1_val[['pop_alvo', 'domicilios_alvo', 'renda_media', 'potencial']].copy()
X1_train_pa.head(1)

,pop_alvo,domicilios_alvo,renda_media,potencial
63,0.03,0.05,0.03,1


### Estatística Univariada

Uma forma um pouco mais sofisticada, é utilizar a classe "SelectKBest" do scikit-learn para selecionar as features.

Na estatística univariada, calcula-se a existência de uma relação estatisticamente significativa entre cada feature e a variável alvo. 

- Serão identificadas as "K" features com o maior P Valor, que provavelmente estão menos relacionadas com a feature alvo. Elas serão eliminadas, e as restantes em "X1_train" serão as selecionadas.
- O método considera apenas cada feature individualmente x a variável alvo. Logo, desconsidera features combinadas que possam ser informativas.

In [120]:
sel_kbest = SelectKBest(score_func=f_regression, k=17) #score_func=f_regression para regressão
sel_kbest.fit_transform(X1_train, y1_train)
features_kbest = list(set(list(X1_train.columns)) - set(list(sel_kbest.get_feature_names_out())))
print(f"Features selecionadas pelo SelectKBest: {features_kbest}")

Features selecionadas pelo SelectKBest: ['domicilios_b2', 'pop_de35a49', 'pop_de50a59', 'populacao']


Mantendo apenas as features selecionadas em "X1_train" "X1_val".

In [121]:
X1_train_kbest = X1_train[features_kbest].copy() 
X1_val_kbest = X1_val[features_kbest].copy() 
X1_train_kbest.head(1)

,domicilios_b2,pop_de35a49,pop_de50a59,populacao
63,0.07,0.03,0.03,0.02


### Baseada em Algoritmos

Outra forma ainda mais sofisticada, é utilizar machine learning para julgar a importância das features, mantendo as mais importantes (model-based feature selection). 

Será utilizada a classe "SelectFromModel" do scikit-learn para isto.
- Neste método, um algoritmo precisa prover alguma medida de importância, para o rankeamento por esta medida.
    - Modelos baseados em árvores proveem "feature_importances_", que encoda a importância de cada feature.
    - Modelos lineares proveem "coefficients", que também podem ser utilizados para isto.
- Será utilizada uma RandomForestRegressor com 100 árvores, e threshhold com um fator de escala multiplicado pela mediana.
- Diferente da estatística univariada, com esta técnica é possível capturar interactions (features derivadas de outras), caso o modelo utilizado seja capáz de capturá-las.

In [166]:
sel_mbfs = SelectFromModel( RandomForestRegressor(n_estimators=10, random_state=0), threshold="0.6*median") 
sel_mbfs.fit(X1_train, y1_train).transform(X1_train)
features_mbfs = sel_mbfs.get_feature_names_out()
features_mbfs

array(['pop_ate9', 'pop_de10a14', 'pop_de25a34', 'pop_mais_de60',
       'pop_alvo', 'domicilios_a1', 'domicilios_a2', 'domicilios_b1',
       'domicilios_c2', 'domicilios_d', 'domicilios_e', 'renda_media'],
      dtype=object)

### Seleção final das Features

Um KNeighborsRegressor com os mesmos parâmetros será utilizado para escolher qual conjunto de features performa melhor. 

Os resultados serão avaliados e o dataset com maior performance será utilizado para treinar o baseline, e demais algoritmos na sequência.

In [258]:
model_dict = {"knn_reg":      ["knn_reg_yhat",       X1_train,       X1_val,       "knn_reg_rmse",       "KNN - mantendo todas as features"],
              "knn_reg_eda":  ["knn_reg_eda_yhat",   X1_train_eda,   X1_val_eda,   "knn_reg_eda_rmse",   "KNN - apenas features com correlação positiva com var. alvo na EDA)" ],
              "knn_reg_pa":   ["knn_reg_pa_yhat",    X1_train_pa,    X1_val_pa,    "knn_reg_pa_rmse",    "KNN - apenas features relacionadas com o público alvo" ],
              "knn_reg_kbest":["knn_reg_kbest_yhat", X1_train_kbest, X1_val_kbest, "knn_reg_kbest_rmse", "KNN - apenas features mais relac. com var. alvo (estatítica univariada) " ],
              "knn_reg_mbfs": ["knn_reg_mbfs_yhat",  X1_train_mbfs,  X1_val_mbfs,  "knn_reg_mbfs_rmse",  "KNN - apenas features mais importantes de RandomForestReg. (model_based)" ]}
fs_res = []
#para cada feature selection
for mod_name, param in model_dict.items():
    #instancia um KNN Regressor com mesmos parâmetros
    mod_name = KNeighborsRegressor(n_neighbors=1)
    #treina e prevê utilizando a feature selection
    param[0] = mod_name.fit(param[1], y1_train).predict(param[2])
    #calcula o erro RMSE da feature selection
    param[3] = ml_error( param[4], y1_val, param[0] )
    fs_res.append( param[3] )
#concatena o retorno de cada feature selection em um mesmo dataframe
fs_res_final = pd.concat(fs_res)

In [259]:
fs_res_final

,Model Name,MAE,MAPE,RMSE
0,KNN - mantendo todas as features,77870.81,0.10,109586.55
0,KNN - apenas features com correlação positiva com var. alvo na EDA),138841.33,0.43,182608.53
0,KNN - apenas features relacionadas com o público alvo,110953.86,0.12,183592.41
0,KNN - apenas features mais relac. com var. alvo (estatítica univariada),206695.71,0.19,366582.59
0,KNN - apenas features mais importantes de RandomForestReg. (model_based),68795.38,0.09,93722.18


Conjunto de features escolhidas: considerando o RMSE, o melhor conjunto de features (de menor erro) foi o baseado em algoritmos (model based selection).

Sobre as métricas utilizadas: No ciclo 1, o R squared foi usado para a avaliação do modelo baseline de regressão, e agora o RMSE foi utilizando. 
- O RMSE informa o quão bem um modelo de regressão pode prever o valor de uma variável resposta em termos absolutos, e se torna mais interessante para comparar a performance de diferentes modelos de regressão linear que o R squared. 
- Interpretação do RMSE (Raiz quadrada do erro médio): A cada predição de faturamento para os bairros, o modelo erro em média ± R$ x,xx.

Os erros MAE e MAPE também foram calculados, podendo ser úteis para reporte ao negócio:
- MAE (Erro absoluto médio): É a diferença entre os valores preditos e reais.
    - Atribui peso igual para todos os erros, logo "dilui o outlier", reduzindo seu peso. O RMSE dá mais peso ao outlier que o MAE, e por isso é utilizado para avaliação dos modelos.
- MAPE (Erro absoluto médio em percentual): É a diferença percentual entre os valores preditos e reais.
    - Interpretação: A cada predição de faturamento para os bairros, o modelo erro em média ± x% na previsão total.


Mantendo apenas as features selecionadas nos demais datasets.

In [167]:
X1_train_mbfs = X1_train[features_mbfs].copy() 
X1_val_mbfs = X1_val[features_mbfs].copy() 
X1_trainval_mbfs = X1_trainval[features_mbfs].copy() 
X1_test_mbfs = X1_test[features_mbfs].copy() 
X1_train_mbfs.head(1)

,pop_ate9,pop_de10a14,pop_de25a34,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_c2,domicilios_d,domicilios_e,renda_media
63,0.01,0.01,0.03,0.04,0.03,0.00,0.01,0.04,0.04,0.02,0.03,0.03


## Implementação de Baseline (Ciclo 1)

Esta seção foi desenvolvida no ciclo 1, e será mantida para comparação da performance obtida contra este ciclo 3.

Como algoritmo de machine learning de base, será implementado o KNN Regressor (K-Nearest Neighbors, ou "K vizinhos mais próximos") da biblioteca scikit-learn.
- É um bom baseline por ser simples de implementar, explicar ao time de negócio, e demandar pouca parametrização.

Funcionamento:
- O KNN não constrói um modelo (é chamado de “lazy”), mas armazena o dataset de treino dentro dele.
- Para prever um novo ponto sem rótulo, encontra o ponto mais próximo dele com rótulo armazenado, sendo seu "nearest neighbor".

Premissas:
- Receber apenas features numéricas, e na mesma escala (demanda pré processamento), pois realiza cálculos de distâncias entre os pontos.

Manter apenas features numéricas, respeitando a premissa do KNN.

In [33]:
X1_train = X1_train.select_dtypes(include=['int64', 'float64'])
X1_val = X1_val.select_dtypes(include=['int64', 'float64'])
X1_train.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e
63,16177,1516,748,1016,1174,2488,3497,2327,3411,0,141,857,1464,1740,1030,87,432


Instanciar um KNN Regressor, treinar com os dados de treino, e realizar predições com os dados de validação.

Será parametrizado neste ciclo um número de k=3, visando um modelo mais simples que o K=5 default, com a expectativa de uma maior capacidade de generalização. Nos próximos ciclos, serão avaliados diferentes valores de K, para comparação.  

In [34]:
knn_reg = KNeighborsRegressor(n_neighbors=3)
knn_reg.fit(X1_train, y1_train)
knn_reg_yhat = knn_reg.predict(X1_val)

A métrica utilizada para medir a performance do modelo será a R squared (ou coeficiente de determinação).
- Trata-se de uma medita estatística de quão próximos os dados estão da linha de regressão ajustada.
- Varia entre entre 0 e 100%, sendo a porcentagem da variação da variável resposta que é explicada por um modelo linear.
- Busca-se portanto, o valor mais próximo de 100% possível, sendo aqui a previsão de faturamento mais próxima da realidade possível.

In [35]:
print("R^2 com dados de Validação {:.4f}".format(knn_reg.score(X1_val, y1_val)))

R^2 com dados de Validação 0.5709


Pode-se calcular também o R^2 a partir da predição do modelo, através do pacote "metrics" do scikit-sklearn.

In [36]:
print(f"R^2 com dados de Validação: {round(r2_score(y1_val, knn_reg_yhat),4)}")

R^2 com dados de Validação: 0.5709


O R Squared do KNN foi satisfatório neste primeiro ciclo. Possivlmente a métrica pode ser melhorada, por meio de:
- Reescala de dados.
- Seleção de features, removendo as não informativas e aquelas não relacionadas ao público alvo.
- Seleção de diferentes valores para K, bem como tunagem de hiperparâmetros no modelo.
- Avaliação da presença de outliers, que podem distorcer os cálculos de distâncias.

Nos próximos ciclos, estes temas serão tratados, e novos algoritmos com diferentes abordagens também serão implementados.

No último ciclo, será avaliada a capacidade de generalização do modelo final (de melhor performance), realizando a previsão com os dados de teste.

## Comparação com o Baseline

Esta seção e as subsequêntes dentro da análise 1, foram implementadas no ciclo 3.

Percebe-se que 3 dos 4 pontos de melhoria identificados no ciclo 1 (seçao anterior) foram sanados. O quarto (seleção de diferentes números de K) será também tratado na sequência.

Mapeamento dos split do dataset já preparados, para facilitar o treinamento de modelos:
- X --> X1_train_mbfs, X1_val_mbfs, X1_trainval_mbfs, X1_test_mbfs
- y --> y1_train,      y1_val,      y1_trainval,      y1_test

Treinar KNN com mesmos parâmetros do ciclo 1.

In [260]:
knn_reg_c2 = KNeighborsRegressor(n_neighbors=3)
knn_reg_c2.fit(X1_train_mbfs, y1_train)
knn_reg_c2_yhat = knn_reg_c2.predict(X1_val_mbfs)

Inspeção manual de Amostras.

In [261]:
print(knn_reg_c2_yhat[:4])
print(y1_val[:4].values)

[ 742261.  750214.  779966. 1902351.]
[ 641865  736698  764380 2119774]


Ao utilizar a métrica R^2, apenas para fins de comparação com o ciclo 1. Na prática, um Rˆ2 maior acompanha um RMSE menor.

In [264]:
print("R^2 com dados de Validação {:.4f}".format(knn_reg_c2.score(X1_val_mbfs, y1_val)))

R^2 com dados de Validação 0.9315


Houve um ganho substancial após as melhorias implementadas. 

Nas próximas seções, serão utilizadas técnicas para melhoria de performance dos modelos, bem como outros modelos também serão experimentados.

## KNN Regressor

A seguir, será implmentado um KNN Regressor com Grid Search + Cross Validation.

O objetivo do Grid Search (uma forma de fine tuning) é encontrar os melhores parâmetros para o modelo, visando uma maior perfomance. 
- Os possíveis parâmetros serão definidos abaixo, e um modelo será gerado para cada possibilidades de combinação pelo Grid Search.

O Cross Validation é um método estatístico que serve para avaliar a performance de modelos, considerando as diferentes possíveis divisões do dataset. 
- Abaixo, será realizada uma divisão de 10 partes no dataset, e obtido o cálculo do score médio de cada parte.

In [356]:
knn_param_grid = {'n_neighbors':[1,2,3,4,5,6,7,8], 'weights':['uniform','distance']}
#instanciar GridSearchCV com KNNRegressor e treinar o modelo
knn_reg_gscv = GridSearchCV(KNeighborsRegressor(), knn_param_grid, cv=10).fit(X1_train_mbfs, y1_train)

Analisar os melhores parâmetros do KNN. 

In [357]:
print(f"Melhores parâmetros: {knn_reg_gscv.best_params_}")

Melhores parâmetros: {'n_neighbors': 3, 'weights': 'distance'}


Predição em cima dos dados de validação.

In [358]:
knn_reg_gscv_yhat = knn_reg_gscv.predict(X1_val_mbfs)

In [359]:
knn_reg_gscva_score = ml_error('knn_reg_gscva', y1_val, knn_reg_gscv_yhat )
knn_reg_gscva_score

,Model Name,MAE,MAPE,RMSE
0,knn_reg_gscva,75002.50,0.17,121608.58


O resultado do KNN com seus melhores parâmetros servirá de base, para comparação com os próximos modelos implementados.

## Regressão Linear - Lasso

Será avaliado na sequência um modelo de regressão linear chamado Lasso.
- Como o dataset tem poucos registros, é imporante regularizar o modelo: reduzir sua complexidade, para evitar overfitting.
- Ele foi escolhido pois assume-se que só algumas features são importantes, logo cabe o uso da regularização L1 (implementada pelo Lasso). O Lasso ignora totalmente algumas features, revelando as mais importantes.
- Serão utilizados os parâmetros:
    - "alpha": controla a força com que os coeficientes são empurrados para zero. Aumentando, reduz-se o overfitting.
    - "max_iter": número máximo de iterações a serem executadas, precisa ser aumentado ao aumentar o alpha.

In [360]:
lasso_param_grid = {'alpha':[1,10,100,1000,10000], 'max_iter':[100000,100000]}
#instanciar GridSearchCV com Lasso e treinar o modelo
lasso_reg_gscv = GridSearchCV(Lasso(random_state=0), lasso_param_grid, cv=10).fit(X1_train_mbfs, y1_train)

Analisar os melhores parâmetros do Lasso. 

In [361]:
print(f"Melhores parâmetros: {lasso_reg_gscv.best_params_}")

Melhores parâmetros: {'alpha': 1000, 'max_iter': 100000}


Predição em cima dos dados de validação.

In [362]:
lasso_reg_gscv_yhat = lasso_reg_gscv.predict(X1_val_mbfs)

In [363]:
lasso_reg_gscva_score = ml_error('lasso_reg_gscva', y1_val, lasso_reg_gscv_yhat)
lasso_reg_gscva_score

,Model Name,MAE,MAPE,RMSE
0,lasso_reg_gscva,106338.48,0.37,164061.91


Modelos lineares são muito poderosos em datasets com muitas features, e tendem a não ter um desempenho tão satisfatória em datasets com poucos registros. É o que foi constatado acima.

Na sequências, um modelo baseado em árvores será utilizado.

## XGBoost Regressor

O XGBoost (Extreme Gradient Boosting) é um ensembles de Decision Trees (árvores de decisão). Ensembles são métodos que combinam múltiplos modelos de machine learning para criar modelos ainda mais poderosos.
- Gradient Boostings estão entre os mais poderosos e amplamente utilizados modelos de ML de aprendizagem supervisionada.
- A principal vantágem do Gradient Boosting é combinar vários modelos simples (weak learners), como swallow trees (árvores rasas). Cada modelo só provê um bom resultado em parte dos dados, mas adicionando mais árvores, melhora-se a performance geral.
- São mais sensíveis a parâmetros que as Random Forests, mas podem prover maior acurácia, se parametrizados adequadamente.
- A escolha dos parâmetros e sua tunagem priorizou o controle de overfitting. 

In [618]:
xgb_param_grid = {'n_estimators':[50], 'max_depth':[4], 'min_child_weight':[4], 'colsample_bytree':[0.5, 0.9], 'eta':[0.1,0,2]}
#instanciar GridSearchCV com XGBRegressor e treinar o modelo
xgb_reg_gscv = GridSearchCV(xgb.XGBRegressor(), xgb_param_grid, cv=10).fit(X1_train_mbfs, y1_train)

Analisar os melhores parâmetros do XGBoost. 

In [619]:
print(f"Melhores parâmetros: {xgb_reg_gscv.best_params_}")

Melhores parâmetros: {'colsample_bytree': 0.5, 'eta': 0.1, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 50}


Predição em cima dos dados de validação.

In [620]:
xgb_reg_gscv_yhat = xgb_reg_gscv.predict(X1_val_mbfs)

In [621]:
xgb_reg_gscva_score = ml_error('xgb_reg_gscva', y1_val, xgb_reg_gscv_yhat)
xgb_reg_gscva_score

,Model Name,MAE,MAPE,RMSE
0,xgb_reg_gscva,68881.02,0.16,102609.99


A performance foi satisfatória.

É possível atingir um RMSE consideravelemnte menor em dados de validação, porém é preciso controlar o overfitting, visto que o objetivo é uma boa performance em dados de teste, e não apenas em dados de validação.

## Performance dos Modelos

A tabela abaixo detalha a melhor performance obtida com cada modelo de machine learning, já com fine tuning e cross validation.

In [622]:
df_an1_scores = pd.concat([knn_reg_gscva_score,lasso_reg_gscva_score,xgb_reg_gscva_score])
df_an1_scores

,Model Name,MAE,MAPE,RMSE
0,knn_reg_gscva,75002.50,0.17,121608.58
0,lasso_reg_gscva,106338.48,0.37,164061.91
0,xgb_reg_gscva,68881.02,0.16,102609.99


O modelo de melhor performance foi o XGBoost, e ele será utilizado para a sequência da análise. 

## Performance de Generalização

Agora, diferente do realizado até aqui, o modelo irá fazer a predição em cima dos dados de teste.

Desta forma, será possível avaliar a sua capacidade de generalização (prever dados inéditos), simulando dados de produção (SP). 

Os parêmetros utilizados serão os mesmos dos utilizados no XGBoost de melhor performance com dados de validação.

In [767]:
xgb_tuning = {'colsample_bytree': 0.5, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 50, 'eta':0.1}
#treina com melhores parâmetros
xgb_reg_gscva_tuned = xgb.XGBRegressor( 
                                        max_depth = xgb_tuning['max_depth'],
                                        min_child_weight = xgb_tuning['min_child_weight'],
                                        n_estimators = xgb_tuning['n_estimators'],
                                        colsample_bytree = xgb_tuning['colsample_bytree'],
                                        eta = xgb_tuning['eta']
                                    ).fit( X1_train_mbfs, y1_train )
#prediz contra teste
xgb_reg_gscva_tuned_yhat = xgb_reg_gscva_tuned.predict( X1_test_mbfs )

In [768]:
xgb_reg_gscva_tuned_final = ml_error('XGBoost Regressor Final', y1_test, xgb_reg_gscva_tuned_yhat )
xgb_reg_gscva_tuned_final

,Model Name,MAE,MAPE,RMSE
0,XGBoost Regressor Final,49556.75,0.18,63924.58


A generalização foi satisfatória considerando o RMSE, que era maior de 100k em validação.

O MAPE ficou levemente acima, porém segue bastante próximo do obtido em validação.

Na próxima seção, o modelo final será treinado.

## Modelo Final

Um modelo final será agora treinado utilizando os dados de X1_trainval (treino + validação), com o melhor set de parâmetros. 

Busca-se com isto aumentar ainda mais a performance do modelo na previsão de dados de produção.

Carregar dados de SP (produção).

In [807]:
df_sp_raw = pd.read_csv('../data/interim/df_sp_raw.csv', index_col=0)
df_sp = df_sp_raw.copy()
print(df_sp.shape)
df_sp.head(1)

(296, 24)


,codigo,bairro,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento,potencial
160,355030251,A. E. Carvalho,São Paulo,SP,94034,12668,6853,9836,7487,14535,21549,10598,10508,0,253,2197,4368,6681,7011,2247,5670,1501,NaN,NaN


Aplicar as mesmas adequações nos dados do df_rj da análise 1 para df_sp.

Derivar features.

In [808]:
df_sp["pop_alvo"] = df_sp["pop_de25a34"] + df_sp["pop_de35a49"]
df_sp["domicilios_alvo"] = df_sp["domicilios_a1"] + df_sp["domicilios_a2"] + df_sp["domicilios_b1"] + df_sp["domicilios_b2"]

Filtrar features.

In [809]:
df_sp = df_sp[features_scaling].copy()
df_sp.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media
160,94034,12668,6853,9836,7487,14535,21549,10598,10508,36084,0,253,2197,4368,6681,7011,2247,5670,6818,1501


Preencher com zero os 3 registros com '-' em renda_media.

In [810]:
df_sp.loc[~df_sp.renda_media.str.isnumeric(), 'renda_media'] = 0

Aplicar preperação nos dados com scaler já pronto.

In [813]:
df_sp[features_scaling] = scaler1.transform(df_sp)
df_sp.head(1)

,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,domicilios_alvo,renda_media
160,0.14,0.10,0.12,0.16,0.12,0.17,0.20,0.13,0.11,0.19,0.00,0.02,0.11,0.21,0.20,0.27,0.45,0.38,0.14,0.01


Manter mesmas features de X1_trainval_mbfs

In [821]:
df_sp.columns

Index(['pop_ate9', 'pop_de10a14', 'pop_de25a34', 'pop_mais_de60', 'pop_alvo',
       'domicilios_a1', 'domicilios_a2', 'domicilios_b1', 'domicilios_c2',
       'domicilios_d', 'domicilios_e', 'renda_media'],
      dtype='object')

In [822]:
X1_trainval_mbfs.columns

Index(['pop_ate9', 'pop_de10a14', 'pop_de25a34', 'pop_mais_de60', 'pop_alvo',
       'domicilios_a1', 'domicilios_a2', 'domicilios_b1', 'domicilios_c2',
       'domicilios_d', 'domicilios_e', 'renda_media'],
      dtype='object')

In [823]:
df_sp = df_sp[['pop_ate9', 'pop_de10a14', 'pop_de25a34', 'pop_mais_de60', 'pop_alvo',
       'domicilios_a1', 'domicilios_a2', 'domicilios_b1', 'domicilios_c2',
       'domicilios_d', 'domicilios_e', 'renda_media']]
df_sp.head(1)

,pop_ate9,pop_de10a14,pop_de25a34,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_c2,domicilios_d,domicilios_e,renda_media
160,0.10,0.12,0.17,0.11,0.19,0.00,0.02,0.11,0.27,0.45,0.38,0.01


In [824]:
X1_trainval_mbfs.head(1)

,pop_ate9,pop_de10a14,pop_de25a34,pop_mais_de60,pop_alvo,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_c2,domicilios_d,domicilios_e,renda_media
37,0.05,0.05,0.08,0.09,0.09,0.00,0.00,0.05,0.16,0.15,0.18,0.01


Treinamento de modelo final com o melhor set de parâmetros, utilizando os dados treino + validação.

Ele fará a previsão dos dados de produção (df_sp).

In [825]:
xgb_tuning = {'colsample_bytree': 0.5, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 50, 'eta':0.1}
#treina com melhores parâmetros
xgb_reg_gscva_tuned_prod = xgb.XGBRegressor( 
                                        max_depth = xgb_tuning['max_depth'],
                                        min_child_weight = xgb_tuning['min_child_weight'],
                                        n_estimators = xgb_tuning['n_estimators'],
                                        colsample_bytree = xgb_tuning['colsample_bytree'],
                                        eta = xgb_tuning['eta']
                                    ).fit( X1_trainval_mbfs, y1_trainval )
#prediz com dados de produção (df_sp)
xgb_reg_gscva_production_yhat = xgb_reg_gscva_tuned_prod.predict(df_sp)

Unificar a previsão com o dataset SP original. 

In [829]:
df_sp_raw['faturamento'] = xgb_reg_gscva_production_yhat

O dataset SP com as com as 296 previsões de faturamento por bairro será exportado, para ser complementado com as análises 2 e 3.

In [833]:
print(df_sp_raw.shape)
df_sp_raw.head(3)

(296, 24)


,codigo,bairro,cidade,estado,populacao,pop_ate9,pop_de10a14,pop_de15a19,pop_de20a24,pop_de25a34,pop_de35a49,pop_de50a59,pop_mais_de60,domicilios_a1,domicilios_a2,domicilios_b1,domicilios_b2,domicilios_c1,domicilios_c2,domicilios_d,domicilios_e,renda_media,faturamento,potencial
160,355030251,A. E. Carvalho,São Paulo,SP,94034,12668,6853,9836,7487,14535,21549,10598,10508,0,253,2197,4368,6681,7011,2247,5670,1501,209502.50,NaN
161,35503020,Aclimação,São Paulo,SP,32791,2297,1017,2096,2197,5341,7281,4917,7645,1413,1734,3704,2351,1946,827,291,1617,5920,1508728.38,NaN
162,355030285,Adventista,São Paulo,SP,104193,15070,7343,10631,8657,17749,23364,11567,9812,0,0,1423,4875,8595,10082,3111,5776,1284,165232.69,NaN


In [834]:
df_sp_raw.to_csv('../data/interim/df_sp_an1_done.csv')